# $k$-Means on a Synthetic Dataset

<div class="alert alert-block alert-success">
<b>Goals:</b> 

* Demonstrate $k$-means clusterings.
* Application of artificial data.
* Focus on technical aspects rather than results and interpretation.
</div>


<div class="alert alert-block alert-info">
<b>Content:</b> In this notebook, we test different k-clusterings on a synthetic dataset.
</div>


__Goals__
* run different versions of k-Means on a synthetic dataset
* analyze outcome and performance with different parametrizations

__Attribution__
* Experiments are based on the [k-means demo](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html) by author: Author: Phil Roth <mr.phil.roth@gmail.com> under License BSD 3 clause
* Parts of the code have been modified and extended to fit the context of the lecture

In [ ]:
# Original Licence Remark (k-Means)
# Author: Phil Roth <mr.phil.roth@gmail.com>
# License: BSD 3 claus

## Create a Synthetic Dataset

In [ ]:
import time   # measure the run time of k-Means
import numpy as np # handle arrays/vectors
import matplotlib.pyplot as plt # plot
import pandas as pd

from sklearn.datasets import make_blobs  # create an artificial dataset
from sklearn.cluster import KMeans  # k-Means
from sklearn.cluster import MiniBatchKMeans # A sometimes faster version of k-Means

In [ ]:
n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state, center_box=(-5.0, 5.0))

In [ ]:
X

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
plt.scatter(X[:,0], X[:,1])

## Run k-Means++

In [ ]:
kmeans_pp = KMeans(
    n_clusters=3, # find 3 clusters
    random_state=random_state, # make experiments replicable
    n_init=20, # find the best clustering out of 20 tries with different initialization
    init='k-means++', # use k-means++ initialization
    max_iter=300, tol=0.0001) ## abort after 300 iterations or if the cost function does not change more than tol

While we run the algorithm, we will also measure the runtime.

In [ ]:
time.time()

In [ ]:
time.time()/60/60/24/365 # time.time() returns the seconds passed since 01.01.1970 0:00:00 (epoch time)

<div class="alert alert-block alert-info">
<b>Technical Remark:</b> Time can be measured using the time module. Time is always measured in reference to 01.01.1970.
    
Keep in mind: measuring runtimes is difficult due to
* the multiple processes running in (pseudo-)parallel on a computer
* the influence of the datasets properties
* the influence of random in the algorithm

Remedies:
* run on isolated machines (to the extent that isolation is possible)
* count time over multiple runs (e.g. using different random-seeds)
* count time over different datasets (e.g. created using different random-seeds)
* repeat the exact same experiment (same parameter, same data) and use the __lowest__ time
</div>

In [ ]:
start_time=time.time()
cluster_assignments=kmeans_pp.fit_predict(X)
print(time.time()-start_time)

In [ ]:
def k_means_report(kmeans):
    print(f'k-means \n * has seen {kmeans.n_features_in_} features,\n \
* used {kmeans.n_iter_} iterations, and \n \
* resulted in an inertia of {kmeans.inertia_}.')

In [ ]:
k_means_report(kmeans_pp)

<div class="alert alert-block alert-info">
<b>Technical Remark:</b> Is the resulting inertia high or low? - Inertia is only useful for comparing two clusterings on the same data using the same number of clusters!
</div>

## Inspect the Resulting Clustering

In [ ]:
kmeans_pp.cluster_centers_

In [ ]:
cluster_assignments

In [ ]:

colors=['darkorange', 'darkmagenta', 'dodgerblue']

def print_clustering(X, kmeans, cluster_assignments):
    plt.figure(figsize=(8, 8))

#    for i in range(0,len(np.unique(cluster_assignments))):
    for i in np.unique(cluster_assignments):
        X_sub=X[cluster_assignments==i, :]
        plt.scatter(X_sub[:, 0], X_sub[:, 1], c=colors[i], label=i)
    
    plt.scatter(
        kmeans.cluster_centers_[:, 0], 
        kmeans.cluster_centers_[:, 1],
        s=350, marker='*', c='crimson', edgecolor='black'
    )

    plt.legend()
    
print_clustering(X, kmeans_pp, cluster_assignments)

## Interpretation
* k-means was used to create 3 clusters
* the three clusters can be visualized (after all our data has only two features)
* the three clusters appear as compact shapes around their centers
* we do not see a clear separation between the clusters

## Evaluation
In the rare case where we already know a valid grouping of the data, we can check wether a clustering algorithm -- unaware of that grouping -- would retrieve the same groups.

Note: This is by definition not an exploratory task. However, it is a way to test clustering algorithms on known data to determine whether they are suitable for exploration of unknown data.

In [ ]:
crosstab=pd.crosstab(y, cluster_assignments)
crosstab.index.name="y"
crosstab.columns.name="clusters"
crosstab

## Compare k-Means Versions
Let's compare k-means++ to the k-means without sophisticated init

In [ ]:
kmeans_random = KMeans(
    n_clusters=3, 
    random_state=random_state, 
    n_init=20, 
    init='random',  # the only difference: Random initialization instead of k-means++
    max_iter=300, tol=0.0001)

In [ ]:
start_time=time.time()
cluster_assignments=kmeans_random.fit_predict(X)
print(time.time()-start_time)

In [ ]:
print(f'k-means has seen {kmeans_random.n_features_in_} features,\n \
used {kmeans_random.n_iter_} iterations, and \n \
resulted in an inertia (TDˆ2) of {kmeans_random.inertia_}.')

In [ ]:
kmeans_random.cluster_centers_

In [ ]:
print_clustering(X, kmeans_random, cluster_assignments)

Observation: The clustering looks similar, the colors are different. Why?

The initial clusterings are determined at random. Thus the cluster indexes (0,1,2) are also randomly distributed.

## Alternative (sometimes faster) Version
MiniBatchKMeans - instead of centroid updates after each sample, centroids are updated after a batch of samples.

In [ ]:
mbk=MiniBatchKMeans(n_clusters=3, init='k-means++', max_iter=100, batch_size=100, random_state=1, n_init=20)

In [ ]:
start_time=time.time()
cluster_assignments=mbk.fit_predict(X)
print(time.time()-start_time)

In [ ]:
mbk.get_params()

In [ ]:
mbk.n_iter_

In [ ]:
mbk.inertia_

In [ ]:
print_clustering(X, mbk, cluster_assignments)

<div class="alert alert-block alert-info">
<b>Take Aways:</b> 

* In this notebook, we have tested different versions of k-means clusterings.
* The difference was mostly in terms of run time.
* Proper visualization is difficult, we were lucky that our data was two-dimensional.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> 
In this notebook, the interpretation of the clusters is missing (makes little sense for artificial data).
</div>


<div class="alert alert-block alert-success">
<b>Play with:</b> 

* Create other datasets that are more or less easily separable.
* Try different $k$ -- as one would without previous knowledge on the expected number of groups.
* Load a real dataset and run $k$-means. Which difficulties do you observe?
</div>
